In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [10]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homes')
#start_time = time.time()

#spark.sql("""select date_built,sum(bedrooms), count(*) from bedrooms group by 1""").show()

#print("--- %s seconds ---" % (time.time() - start_time))


In [20]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
  SELECT
    date_built,
    ROUND(avg(price), 2)
  FROM homes
  WHERE
    bedrooms = 4
  GROUP BY date_built
  ORDER BY date_built DESC
  """).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2017|           296576.69|
|      2016|           296050.24|
|      2015|           307908.86|
|      2014|           299073.89|
|      2013|           299999.39|
|      2012|           298233.42|
|      2011|            302141.9|
|      2010|           296800.75|
+----------+--------------------+



In [22]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
  SELECT
    date_built,
    ROUND(avg(price), 2)
  FROM homes
  WHERE
    bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY date_built
  ORDER BY date_built DESC
  """).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2017|           280317.58|
|      2016|            293965.1|
|      2015|           297609.97|
|      2014|           298264.72|
|      2013|           303676.79|
|      2012|           307539.97|
|      2011|           276553.81|
|      2010|           285010.22|
+----------+--------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
  SELECT
    date_built,
    ROUND(avg(price), 2)
  FROM homes
  WHERE
    bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
  ORDER BY date_built DESC
  """).show()


In [27]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
  SELECT
    view,
    ROUND(avg(price), 2)
  FROM homes
  WHERE
    bedrooms = 3 AND bathrooms = 3 AND price >= 250000
  GROUP BY view
  ORDER BY view DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|            840349.0|
|  98|            649815.0|
|  97|            639842.0|
|  96|            818937.0|
|  95|            812657.0|
|  94|            511412.0|
|  93|            684082.0|
|  92|            918640.0|
|  91|            944188.0|
|  90|            662334.5|
|   9|           347633.89|
|  89|            791915.0|
|  88|            758324.0|
|  86|            547546.5|
|  84|            851282.5|
|  83|            996773.0|
|  81|            584004.5|
|  80|            799281.5|
|   8|           355305.52|
|  79|            864097.0|
+----+--------------------+
only showing top 20 rows

--- 0.7205212116241455 seconds ---


In [28]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homes")


DataFrame[]

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [30]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
  SELECT
    view,
    ROUND(avg(price), 2)
  FROM homes
  WHERE
    bedrooms = 3 AND bathrooms = 3 AND price >= 250000
  GROUP BY view
  ORDER BY view DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|            840349.0|
|  98|            649815.0|
|  97|            639842.0|
|  96|            818937.0|
|  95|            812657.0|
|  94|            511412.0|
|  93|            684082.0|
|  92|            918640.0|
|  91|            944188.0|
|  90|            662334.5|
|   9|           347633.89|
|  89|            791915.0|
|  88|            758324.0|
|  86|            547546.5|
|  84|            851282.5|
|  83|            996773.0|
|  81|            584004.5|
|  80|            799281.5|
|   8|           355305.52|
|  79|            864097.0|
+----+--------------------+
only showing top 20 rows

--- 0.6453125476837158 seconds ---


In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")


In [32]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('delayed_partitioned')

In [33]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('dates')

In [34]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""
  SELECT
    view,
    ROUND(avg(price), 2)
  FROM dates
  WHERE
    bedrooms = 3 AND bathrooms = 3 AND price >= 250000
  GROUP BY view
  ORDER BY view DESC
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|            840349.0|
|  98|            649815.0|
|  97|            639842.0|
|  96|            818937.0|
|  95|            812657.0|
|  94|            511412.0|
|  93|            684082.0|
|  92|            918640.0|
|  91|            944188.0|
|  90|            662334.5|
|   9|           347633.89|
|  89|            791915.0|
|  88|            758324.0|
|  86|            547546.5|
|  84|            851282.5|
|  83|            996773.0|
|  81|            584004.5|
|  80|            799281.5|
|   8|           355305.52|
|  79|            864097.0|
+----+--------------------+
only showing top 20 rows

--- 1.9650952816009521 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homes")

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("homes")

False